In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import pandas as pd
import backtrader as bt
from datetime import datetime
import os
import sys
import json
from backtrader.analyzers import Returns,DrawDown,SharpeRatio,TradeAnalyzer

In [2]:
sys.path.append('../backtesting/')
from strategies import TestStrategy, SMA_RSI, MovingAverageCrossStrategy

In [3]:
def prepare_cerebro(asset,strategy,data_path,start_date:str,end_date:str=datetime.now(),cash:int=100,commission:float=0)->bt.Cerebro:
    if end_date ==None:
        end_date= datetime.strftime(datetime.now(),"%Y-%m-%d")

    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    cerebro.broker.setcommission(commission=commission)
    cerebro.addstrategy(strategy)
    
    isExist = os.path.exists(data_path)
    if not isExist:
        data= yf.download(asset,start_date,end=end_date)
        data.to_csv(data_path)
    
    data = bt.feeds.YahooFinanceCSVData(
        dataname=data_path,
        fromdate=datetime.strptime(start_date,"%Y-%m-%d"),
        todate=datetime.strptime(end_date,"%Y-%m-%d"),
        reverse=False
    )

    cerebro.adddata(data)
    # cerebro.addanalyzer(AnnualReturn)
    cerebro.addanalyzer(TradeAnalyzer)
    return cerebro

In [4]:
def run_test(cerebro:bt.Cerebro):

    result={}

    cerebro.addanalyzer(SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(Returns, _name='returns')
    cerebro.addanalyzer(DrawDown, _name='draw')
    cerebro.addanalyzer(TradeAnalyzer, _name='trade')
    
    starting = cerebro.broker.getvalue()
    res=cerebro.run()
    final=cerebro.broker.getvalue()

    thestrat = res[0]

    sharpe=thestrat.analyzers.sharpe.get_analysis()
    return_val=thestrat.analyzers.returns.get_analysis()
    drawdown=thestrat.analyzers.draw.get_analysis()
    trade=thestrat.analyzers.trade.get_analysis()

    result["sharpe_ratio"]=sharpe['sharperatio']
    result["return"]=return_val['rtot']
    result['max_drawdown'] = drawdown['max']['drawdown']
    
    try:
        result['win_trade']=trade['won']['total']
    except:
        result['win_trade']="Undefined"
    
    try:
        result['loss_trade']=trade['lost']['total']
    except:
        result['loss_trade']="Undefined"

    try:
        result['total_trade']=trade['total']['total']
    except:
        result['total_trade']="Undefined"

    result['start_portfolio']=starting
    result['final_portfolio']=final

    return result

In [5]:
cerebro=prepare_cerebro('BITC',SMA_RSI,"../data/BITC.csv","2023-06-21","2024-06-21")

result=run_test(cerebro)
result

{'sharpe_ratio': 7.29638656907243,
 'return': 0.15896763109564546,
 'max_drawdown': 11.43481996407093,
 'win_trade': 8,
 'loss_trade': 9,
 'total_trade': 17,
 'start_portfolio': 100,
 'final_portfolio': 117.22999999999999}

### SMA_RSI Strategy Metrics 
#### Sharpe ratio
- The sharpe ratio at 7.30 is outstanding (generally a Sharpe ratio above 1 is considered adequate/good, above 2 is very good, and above 3 is excellent)
#### Return 
- Indicates a 15.90% return on the initial investment over the backtesting period, indicating a profitable strategy
#### Max Drawdown
- Indicates that the portfolio experienced a decline of up to 11.43% from its highest value
#### Trades
- Of the 17 trades executed, 8 were profitable and 9 were unprofitable.
#### Portfolio
- The backtest started with a portfolio value of 100 units and ended with 117.23 units, showing overall growth

In [6]:
cerebro=prepare_cerebro('BITC',TestStrategy,"../data/BITC.csv","2023-06-21","2024-06-21")

result=run_test(cerebro)
result

{'sharpe_ratio': 2.922273238765637,
 'return': 0.5116253039365551,
 'max_drawdown': 21.90520735890242,
 'win_trade': 'Undefined',
 'loss_trade': 'Undefined',
 'total_trade': 1,
 'start_portfolio': 100,
 'final_portfolio': 166.8}

### Test Strategy Metrics
#### Sharpe ratio
- The sharpe ratio at 1.93 is quite good, indicates that the strategy provides good returns relative to the risk taken.
#### Return 
- Indicates a 10.88% return on the initial investment over the backtesting period.
#### Max Drawdown
- Indicates that at its worst point, the portfolio value declined by 8.96% from its highest value before recovering.
#### Trades
- Of the 21 trades executed, 12 were profitable and 9 were unprofitable.
#### Portfolio
- The backtest started with a portfolio value of 100 units and ended with 111.49 units, showing overall growth

In [7]:
cerebro=prepare_cerebro('ETH-USD',TestStrategy,"../data/ETH-USD.csv","2023-06-22","2024-06-22")

result=run_test(cerebro)
result

{'sharpe_ratio': None,
 'return': 0.0,
 'max_drawdown': 0.0,
 'win_trade': 'Undefined',
 'loss_trade': 'Undefined',
 'total_trade': 0,
 'start_portfolio': 100,
 'final_portfolio': 100.0}